### Similarity search using Oracle Vector Store

In [7]:
import logging
from pprint import pprint

import oracledb

# to compute embeddings vectors
from oci_cohere_embeddings_utils import OCIGenAIEmbeddingsWithBatch

from langchain_community.vectorstores.oraclevs import OracleVS
from langchain_community.vectorstores.utils import DistanceStrategy

from utils import load_configuration

# private information
from config_private import COMPARTMENT_ID, DB_USER, DB_PWD, DB_HOST_IP, DB_SERVICE

#### Settings

In [8]:
# Configure logging
logger = logging.getLogger("ConsoleLogger")

logger.setLevel(logging.INFO)

# load the config in a toml file
config = load_configuration()

# embeddings model: we're using OCI GenAI multilingual Cohere
OCI_EMBED_MODEL = config["embeddings"]["oci"]["embed_model"]
EMBED_ENDPOINT = config["embeddings"]["oci"]["embed_endpoint"]

# number of docs retrieved for each query
# reduced from config to simplify outpute here
TOP_K = 4

# to connect to DB
# if you don't change, the port is 1521
DSN = f"{DB_HOST_IP}:1521/{DB_SERVICE}"

print("The general configuration is:")
print()
pprint(config)

The general configuration is:

{'embeddings': {'cohere': {},
                'embed_model_type': 'OCI',
                'oci': {'embed_batch_size': 90,
                        'embed_endpoint': 'https://inference.generativeai.us-chicago-1.oci.oraclecloud.com',
                        'embed_model': 'cohere.embed-multilingual-v3.0'}},
 'llm': {'cohere': {'llm_model': 'command-r'},
         'max_tokens': 1024,
         'model_type': 'OCI',
         'oci': {'endpoint': 'https://inference.generativeai.us-chicago-1.oci.oraclecloud.com',
                 'llm_model': 'meta.llama-2-70b-chat'},
         'temperature': 0.1},
 'reranker': {'add_reranker': True,
              'cohere_reranker_model': 'rerank-multilingual-v3.0'},
 'retriever': {'top_k': 8, 'top_n': 4},
 'text_splitting': {'books_dir': './books',
                    'chunk_overlap': 50,
                    'chunk_size': 1500},
 'tracing': {'enable': False, 'langchain_project': 'workshop-1'},
 'ui': {'add_references': True,
        

In [9]:
# utility function
def print_metadata(v_metadata):
    """
    this is the format:
    {'source': './books/oracle-ai-vector-search-users-guide.pdf', 'page': 0}
    """
    print(f"- Source: {v_metadata['source']}, page: {v_metadata['page']}")

In [15]:
# create client for Embeddings and AI Vector Search

# Embed model here is needed to embed the query!
# for embeddings we're using the extension that handles batching
embed_model = OCIGenAIEmbeddingsWithBatch(
    auth_type="API_KEY",
    model_id=OCI_EMBED_MODEL,
    service_endpoint=EMBED_ENDPOINT,
    compartment_id=COMPARTMENT_ID,
)

try:
    # we need to provide a connection as input to OracleVS
    connection = oracledb.connect(user=DB_USER, password=DB_PWD, dsn=DSN)
    logger.info("Connection successful!")

    # get an instance of OracleVS
    v_store = OracleVS(
        client=connection,
        table_name="ORACLE_KNOWLEDGE",
        distance_strategy=DistanceStrategy.COSINE,
        embedding_function=embed_model,
    )

except Exception as e:
    logger.error("Connection failed!")
    logger.error(e)

2024-06-06 13:01:53,032 - INFO - Connection successful!


In [16]:
# create a retriever from the Vector Store
retriever = v_store.as_retriever(search_kwargs={"k": TOP_K})

logger.info("Retriever created...")

2024-06-06 13:01:54,160 - INFO - Retriever created...


In [17]:
%%time
question = "What is AI Vector Search??"

result_docs = retriever.invoke(question)

CPU times: user 58.8 ms, sys: 41 ms, total: 99.8 ms
Wall time: 595 ms


In [18]:
# display results

print("")
print("--- Document retrieved from the knowledge base ---")
print()

for i, doc in enumerate(result_docs):
    print("-------------------------------------------")
    print(f"Document n. {i+1}")
    print("")
    print("- Content:")
    print(doc.page_content)
    print("")
    print_metadata(doc.metadata)
    print("-------------------------------------------")
    print("")


--- Document retrieved from the knowledge base ---

-------------------------------------------
Document n. 1

- Content:
Oracle® Database
Oracle AI Vector Search User's Guide
23ai
F87786-01
May 2024

- Source: ./books/oracle-ai-vector-search-users-guide.pdf, page: 0
-------------------------------------------

-------------------------------------------
Document n. 2

- Content:
Manage the Different Categories of Vector Indexes
5-8

- Source: ./books/oracle-ai-vector-search-users-guide.pdf, page: 148
-------------------------------------------

-------------------------------------------
Document n. 3

- Content:
1
Overview
Oracle AI Vector Search stores and indexes vector embeddings for fast retrieval and
similarity search.
•Overview of Oracle AI Vector Search
Oracle AI Vector Search is designed for Artificial Intelligence (AI) workloads and allows
you to query data based on semantics, rather than keywords.
•Why Use Oracle AI Vector Search?
One of the biggest benefits of Oracle AI V